# Programación declarativa @ URJC
# Programación funcional
## Curso 22-23, convocatoria ordinaria (31 de octubre de 2022)
## Campus de Móstoles


# Definiciones auxiliares

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Algunas definiciones de tipos y funciones auxiliares

In [2]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

object Tree{
    def void[A]: Tree[A] = Empty()
    def leaf[A](a: A): Node[A] = Node(Empty(), a, Empty())
    def right[A](a: A, tree: Tree[A]): Node[A] = Node(Empty(), a, tree)
    def left[A](tree: Tree[A], a: A): Node[A] = Node(tree, a, Empty())
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = Node(left, a, right)
}

import Tree._

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

defined class Tree
defined class Empty
defined class Node
defined object Tree
import Tree._


defined function foldTree

In [3]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
 
        // Returns the number of elements of this list
        def length: Int
        def reverse: List[A]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard y Tipos Algebraicos de Datos

In [1]:
trait Isomorphic[A, B]{
    
    def from(a: A): B
    
    def to(b: B): A

}

defined trait Isomorphic

In [5]:
type Not[P] = P => Nothing

defined type Not

# Ejercicio 1 (a)
__(2 puntos)__

__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la validez del siguiente argumento lógico: 

$\{ p \rightarrow (q \vee r), \neg r, q \rightarrow (s \vee t), \neg (s \vee t)\} \vdash \neg p $

In [9]:
def proof[P, Q, R, S, T](p1: P => Either[Q, R], 
                         p2: Not[R], 
                         p3: Q => Either[S, T], 
                         p4: Not[Either[S, T]]): 
                             Not[P] = 
     (p: P) => 
        p1(p) match {    
            case Left(q) => p4(p3(q))
            case Right(r) => p2(r)
        }

defined function proof

__b) (1 punto)__ Utiliza la correspondencia Curry-Howard para demostrar la siguiente tautología de la lógica clásica proposicional: $\vdash_c (p \rightarrow q) \vee (q \rightarrow r)$. Para ello, utiliza como premisa adicional la ley del tercio excluso aplicada a la proposición $q$, es decir, demuestra la validez intuicionista del siguiente argumento lógico:

$\{ q \vee \neg q \} \vdash (p \rightarrow q) \vee (q \rightarrow r)$

In [10]:
def proof[P, Q, R](lemq: Either[Q, Not[Q]]): Either[P => Q, Q => R] = 
    lemq match {
        case Left(q) => 
            Left(_ => q)
        case Right(nq) => 
            Right(nq)
    }

defined function proof

# Ejercicio 1 (b)
__(2 puntos)__

__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la validez del siguiente argumento lógico: 

$ \{ p \rightarrow q, r \vee s, s \rightarrow \neg q, \neg r \} \vdash \neg p $

In [11]:
def proof[P, Q, R, S](p1: P => Q, 
                      p2: Either[R, S], 
                      p3: S => Not[Q], 
                      p4: Not[R]): Not[P] = 
    p => p2 match {
        case Left(r) => p4(r)
        case Right(s) => p3(s)(p1(p))
    }

defined function proof

__b) (1 punto)__ Utiliza la correspondencia Curry-Howard para demostrar la siguiente tautología de la lógica clásica proposicional: $\vdash_c \neg (p \rightarrow q) \rightarrow (p \wedge \neg q) $. Para ello, utiliza como premisa adicional la ley del tercio excluso aplicada a la proposición $p$, es decir, demuestra la validez intuicionista del siguiente argumento lógico:

$\{ p \vee \neg p \} \vdash \neg (p \rightarrow q) \rightarrow (p \wedge \neg q) $

In [12]:
def proof[P, Q](lemp: Either[P, Not[P]]): Not[P => Q] => (P, Not[Q]) = 
    (npq: (P => Q) => Nothing) => 
        (lemp match {
            case Left(p) => p
            case Right(np) => npq(np)
        }, 
         q => npq(_ => q)
        )

defined function proof

# Ejercicio 2
__(2 puntos)__

Desde un punto de vista algebraico, una lista de elementos de tipo `A` puede representarse como el siguiente tipo algebraico de datos recursivo:

$List[A] = 1 + A * List[A] $

Si expandimos la definición del tipo lista en la parte derecha obtenemos lo siguiente:

$List[A] = 1 + A * (1 + A * List[A]) = 1 + A + A^2 * List[A]$

Se pide demostrar que los tipos $List[A]$ y $1 + A + A^2 * List[A]$ son isomorfos.

In [14]:
class Iso[A] extends Isomorphic[Either[Unit, Either[A, (A, A, List[A])]], List[A]]{
    
    def from(e: Either[Unit, Either[A, (A, A, List[A])]]): List[A] = 
        e match {
            case Left(_) => Nil
            case Right(Left(a)) => a :: Nil
            case Right(Right((h1, h2, t))) => h1 :: h2 :: t
        }
    
    def to(l: List[A]): Either[Unit, Either[A, (A, A, List[A])]] = 
        l match {
            case Nil => Left(()) 
            case a :: Nil => Right(Left(a))
            case h1 :: h2 :: t => Right(Right((h1, h2, t)))
        }
}

defined class Iso

# Ejercicio 3
__(3 puntos)__

Considérense las funciones siguientes, `parse`, `validate` e `ints`, las cuales hacen uso de las funciones auxiliares `parseNumber`, `validateNumber` y `doubleToInt`:


In [17]:
def parseNumber(s: String): Either[String, Int] = 
    s.toIntOption match {
        case None => Left("'" + s + "' is not a number")
        case Some(i) => Right(i)
    }

def parse(l: List[String]): Either[String, List[Int]] =
    l match {
        case Nil => Right(Nil)
        case head :: tail => 
            val tailParse: Either[String, List[Int]] = parse(tail)
            val headInt: Either[String, Int] = parseNumber(head)
            (headInt, tailParse) match {
                case (Right(h), Right(t)) => Right(h :: t)
                case (Right(_), Left(s)) => Left(s)
                case (Left(s), _) => Left(s)
            }
    }


defined function parseNumber
defined function parse

In [ ]:
def validateNumber(i: Int): Either[String, Int] = 
    if (i < 0) Left("'" + i + "' is negative") 
    else Right(i)

def validate(l: List[Int]): Either[String, List[Int]] = 
    l match {
        case Nil => Right(Nil)
        case head :: tail => 
            val tailValidate: Either[String, List[Int]] = validate(tail)
            val headInt: Either[String, Int] = validateNumber(head)
            (headInt, tailValidate) match {
                case (Right(h), Right(t)) => Right(h :: t)
                case (Right(_), Left(s)) => Left(s)
                case (Left(s), _) => Left(s)
            }
    }

In [ ]:
def doubleToInt(d: Double): Either[String, Int] = 
    if (!d.isValidInt) Left("The fractional part of '" + d + "' is not zero") 
    else Right(d.toInt)

def ints(l: List[Double]): Either[String, List[Int]] = 
    l match {
        case Nil => Right(Nil)
        case head :: tail => 
            val tailInts: Either[String, List[Int]] = ints(tail)
            val headInt: Either[String, Int] = doubleToInt(head)
            (headInt, tailInts) match {
                case (Right(h), Right(t)) => Right(h :: t)
                case (Right(_), Left(s)) => Left(s)
                case (Left(s), _) => Left(s)
            }
    }

La función `parse` traduce todas las cadenas de caracteres de una lista a sus representaciones numéricas correspondientes. Si alguna de las cadenas no representa un número, entonces la función devuelve un valor de tipo `Left` indicando la primera cadena (más a la izquierda) que no cumple dicha condición. Por ejemplo: 

In [ ]:
parse(List("1", "2", "3", "4")) == Right(List(1, 2, 3, 4))
parse(List("1", "2", "aaa", "bb")) == Left("'aaa' is not a number")

La función `validate` comprueba que todos los números de una lista son mayores o iguales a cero. Si se cumple esta condición, entonces la función devuelve la misma lista de entrada; en otro caso, devuelve un valor de tipo `Left` indicando el primer número (más a la izquierda) que no cumple la condición.

In [ ]:
validate(List(0,1,2,3,4)) == Right(List(0, 1, 2, 3, 4))
validate(List(0,1,-2)) == Left("'-2' is negative")

La función `ints` comprueba que todos los números de tipo `Double` (equivalentes al tipo `double` de Java) son números enteros, es decir, que su parte fraccional es cero (ese es el cometido de la función `_.isValidInt`). Si se cumple esta condición, entonces la función devuelve la conversión a enteros de los números de tipo `Double` (realizada mediante la función `_.toInt`); en otro caso, devuelve un valor de tipo `Left` indicando el primer número (más a la izquierda) de tipo `Double` que no cumple la condición. Por ejemplo: 

In [ ]:
ints(List(1.0, 2.0, 3.0, 4.0, 5.0)) == Right(List(1, 2, 3, 4, 5))
ints(List(1.0, 2.2, 3.333, 4.3)) == Left("The fractional part of '2.2' is not zero")

El procesamiento en todos estos casos es similar: tenemos una función que se aplica a cada elemento de la lista, y que podría resultar exitosa o no (el éxito o fracaso se indica mediante un valor de tipo `Either`). Si esta función se puede aplicar con éxito a _todos_ los elementos, se devuelve una lista con todos los resultados; en otro caso, se devuelve un valor de tipo `Left` indicando el primer elemento (más a la izquierda) que genera un error.

__a) (2 puntos)__ Implementa una función de orden superior que encapsule el patrón de diseño descrito anteriormente y utilizado en las funciones `parse` y `validate`. A esta función la denominaremos `traverse`:

In [ ]:
def traverse[A, B, E](l: List[A])(f: A => Either[E, B]): Either[E, List[B]] = 
    l match {
        case Nil => Right(Nil)
        case head :: tail => 
            val tailTraverse: Either[E, List[B]] = traverse(tail)(f)
            val headB: Either[E, B] = f(head)
            (headB, tailTraverse) match {
                case (Right(h), Right(t)) => Right(h :: t)
                case (Right(_), Left(e)) => Left(e)
                case (Left(e), _) => Left(e)
            }
    }

__b) (1 punto)__ Reimplementa las funciones `parse` y `validate` de una manera más modular reutilizando la función de orden superior `traverse` implementada en el apartado anterior. 

In [ ]:
def parse(l: List[String]): Either[String, List[Int]] = 
    traverse(l)(parseNumber)

In [ ]:
def validate(l: List[Int]): Either[String, List[Int]] = 
    traverse(l)(validateNumber)

In [ ]:
def ints(l: List[Double]): Either[String, List[Int]] = 
    traverse(l)(doubleToInt)

# Ejercicio 4
__(3 puntos)__

Dada una función `f: A => B => C`, se desea implementar una función de orden superior `lift1st` que permita transformar `f` en una función similar pero que reciba como primer argumento una lista de `A`s, en lugar de simplemente una `A`. Es decir, `lift1st(f): List[A] => B => List[C]`. La idea es que la expresión `lift1st(f)(l)(b)` consista en una lista que contenga todos los resultados `f(a)(b)` obtenidos tras aplicar `f(_)(b)` a cada uno de los elementos `a` de la lista `l`. A continuación se ilustra el funcionamiento deseado de la función `lift1st` para funciones del tipo `String => Int => Boolean`:

In [6]:
class TestLift1st(lift1st: (String => Int => Boolean) => (List[String] => Int => List[Boolean]))
extends FlatSpec with Matchers{

    def f: String => Int => Boolean = 
        s => i => s.length >= i 

    "lift1st" should "work" in {
        lift1st(f)(List("", "a", "ab", "abc", "abcd"))(3) shouldBe 
            List(f("")(3), f("a")(3), f("ab")(3), f("abc")(3), f("abcd")(3))
        
        lift1st(f)(List("", "a", "ab", "abc", "abcd"))(2) shouldBe 
            List(false, false, true, true, true)
        
        lift1st(f)(List("", "a", "ab", "abc", "abcd"))(1) shouldBe 
            List(false, true, true, true, true)
        
        lift1st(f)(List("", "a", "ab", "abc", "abcd"))(0) shouldBe 
            List(true, true, true, true, true)
    }
}

defined class TestLift1st

In [7]:
class TestLift2nd(lift2nd: (String => Int => Boolean) => (String => List[Int] => List[Boolean]))
extends FlatSpec with Matchers{

    def f: String => Int => Boolean = 
        s => i => s.length >= i 
    
    "lift2nd" should "work" in {
        lift2nd(f)("abc")(List(0,1,2,3,4,5)) shouldBe 
            List(f("abc")(0), f("abc")(1), f("abc")(2), f("abc")(3), f("abc")(4), f("abc")(5))
        
        lift2nd(f)("a")(List(0,1,2,3,4)) shouldBe 
            List(true, true, false, false, false)
        
        lift2nd(f)("")(List(0,1,2)) shouldBe 
            List(true, false, false)
    }
}

defined class TestLift2nd

__a) (1 punto)__ Implementa la función `lift1st` recursivamente.

In [8]:
def lift1st[A, B, C](f: A => B => C)(l: List[A])(b: B): List[C] = 
    l match {
        case Nil => Nil
        case a :: t => 
            val tailSol: List[C] = lift1st(f)(t)(b)
            f(a)(b) :: tailSol
    }

defined function lift1st

In [27]:
def lift1st[A, B, C](f: A => B => C)(l: List[A])(b: B): List[C] = 
    l match {
        case Nil => Nil
        case a :: t => f(a)(b) :: lift1st(f)(t)(b)
    }

defined function lift1st

In [28]:
run(new TestLift1st(lift1st))

cmd5$Helper$TestLift1st:
lift1st
- should work


In [10]:
def lift2nd[A, B, C](f: A => B => C)(a: A)(l: List[B]): List[C] = 
    l match {
        case Nil => Nil
        case b :: t => 
            val tailSol: List[C] = lift2nd(f)(a)(t)
            f(a)(b) :: tailSol
    }

defined function lift2nd

In [11]:
run(new TestLift2nd(lift2nd))

cmd6$Helper$TestLift2nd:
lift2nd
- should work


__b) (1 punto)__ Implementa la función `lift1st` mediante la función de orden superior `foldRight`.

In [12]:
def lift1st[A, B, C](f: A => B => C)(l: List[A])(b: B): List[C] = 
    l.foldRight(List[C]())(
        (a, tailSol) => f(a)(b) :: tailSol
    )

defined function lift1st

In [13]:
run(new TestLift1st(lift1st))

cmd5$Helper$TestLift1st:
lift1st
- should work


In [21]:
def lift2nd[A, B, C](f: A => B => C)(a: A)(l: List[B]): List[C] = 
    l.foldRight(List[C]())(f(a)(_) :: _)

defined function lift2nd

In [22]:
run(new TestLift2nd(lift2nd))

cmd6$Helper$TestLift2nd:
lift2nd
- should work


__c) (1 punto)__ Implementa la función `lift1st` mediante la función de orden superior `map`.

In [ ]:
def lift1st[A, B, C](f: A => B => C)(l: List[A])(b: B): List[C] = 
    l.map(a => f(a)(b))

In [23]:
def lift1st[A, B, C](f: A => B => C)(l: List[A])(b: B): List[C] = 
    l.map(f(_)(b))

defined function lift1st

In [24]:
run(new TestLift1st(lift1st))

cmd5$Helper$TestLift1st:
lift1st
- should work


In [25]:
def lift2nd[A, B, C](f: A => B => C)(a: A)(l: List[B]): List[C] = 
    l.map(f(a))

defined function lift2nd

In [26]:
run(new TestLift2nd(lift2nd))

cmd6$Helper$TestLift2nd:
lift2nd
- should work
